# Solving elasticity problems using JuliaFEM

Author(s): Jukka Aho

**Abstract**: Solving elasticity equations using JuliaFEM.

### Weak form

Given function spaces
\begin{align}
\boldsymbol{\mathcal{U}} & =\left\{ \boldsymbol{u}\in H^{1}\left(\Omega\right)|\boldsymbol{u}\left(\boldsymbol{X},t\right)=\hat{\boldsymbol{u}}\left(\boldsymbol{X},t\right)\text{ on }\Gamma_{\mathrm{u}}\right\} ,\\
\boldsymbol{\mathcal{V}} & =\left\{ \delta\boldsymbol{u}\in H^{1}\left(\Omega\right)|\delta\boldsymbol{u}\left(\boldsymbol{X}\right)=0\text{ on }\Gamma_{\mathrm{u}}\right\} ,
\end{align}
find $\boldsymbol{u}\in\boldsymbol{\mathcal{U}}$ such that
\begin{equation}
\delta\mathcal{W}:=\int_{\Omega_{0}}\rho_{0}\ddot{\boldsymbol{u}}\cdot\delta\boldsymbol{u}\,\mathrm{d}V_{0}+\int_{\Omega_{0}}\boldsymbol{S}:\delta\boldsymbol{E}\,\mathrm{d}V_{0}-\int_{\Omega_{0}}\hat{\boldsymbol{b}}_{0}\cdot\delta\boldsymbol{u}\,\mathrm{d}V_{0}-\int_{\Gamma_{\sigma}}\hat{\boldsymbol{t}}_{0}\cdot\delta\boldsymbol{u}\,\mathrm{d}A_{0} =0 \qquad\forall\delta\boldsymbol{u}\in\boldsymbol{\mathcal{V}}
\end{equation}

### Some formulas
\begin{align}
J & =\det\left(F\right)\\
I_{c} & =\mbox{tr}\left(C\right)\\
\mathbf{C} & =\mathbf{F}^{\mathrm{T}}\mathbf{F}\\
\mathbf{F} & =\mathbf{I}+\nabla\mathbf{u}\\
\mathbf{E} & =\frac{1}{2}\left(\mathbf{F}^{\mathrm{T}}\mathbf{F}-\mathbf{I}\right)
\end{align}

### Potential energy

\begin{equation}
\underset{u\in\boldsymbol{\mathcal{U}}}{\min}\Pi\left(\mathbf{u}\right)
\end{equation}
\begin{equation}
\Pi\left(\mathbf{u}\right)=\int_{\Omega}\psi\left(\mathbf{u}\right)-\int_{\Omega}\hat{\mathbf{b}}_{0}\cdot\mathbf{u}-\int_{\Gamma_{\sigma}}\hat{\mathbf{t}}_{0}\cdot\mathbf{u}\,\mathrm{d}A_{0}
\end{equation}

### Material models

https://en.wikipedia.org/wiki/Strain_energy_density_function

Saint-Venant-Kirchhoff model https://en.wikipedia.org/wiki/Hyperelastic_material
\begin{equation}
\psi\left(\mathbf{E}\right)=\frac{\lambda}{2}\left[\mbox{tr}\left(\mathbf{E}\right)\right]^{2}+\mu\mbox{tr}\left(\mathbf{E}^2\right)
\end{equation}

neo-Hookean material https://en.wikipedia.org/wiki/Neo-Hookean_solid
\begin{equation}
\psi=\frac{\mu}{2}\left(I_{c}-3\right)-\mu\ln\left(J\right)+\frac{\lambda}{2}\ln\left(J\right)^{2}
\end{equation}

In [1]:
using JuliaFEM
using JuliaFEM.Core: Element, Tri3, Tet4, Tri6, Tet10
using JuliaFEM.Core: ElasticityProblem, DirichletProblem
using JuliaFEM.Core: DirectSolver

In [2]:
@time begin
    # Linear models
    #model = open(JuliaFEM.Core.parse_abaqus, "../geometry/piston/piston_8789_P1.inp")
    #model = open(JuliaFEM.Core.parse_abaqus, "../geometry/piston/piston_16436_P1.inp")
    #model = open(JuliaFEM.Core.parse_abaqus, "../geometry/piston/piston_27343_P1.inp")
    #model = open(JuliaFEM.Core.parse_abaqus, "../geometry/piston/piston_45510_P1.inp")
    #model = open(JuliaFEM.Core.parse_abaqus, "../geometry/piston/piston_75470_P1.inp")
    #model = open(JuliaFEM.Core.parse_abaqus, "../geometry/wrench/wrench_128903_P1.inp")

    # Quadratic models
    #model = open(JuliaFEM.Core.parse_abaqus, "../geometry/piston/piston_19611_P2.inp")
    #model = open(JuliaFEM.Core.parse_abaqus, "../geometry/piston/piston_55950_P2.inp")
    #model = open(JuliaFEM.Core.parse_abaqus, "../geometry/piston/piston_107168_P2.inp")
    #model = open(JuliaFEM.Core.parse_abaqus, "../geometry/piston/piston_345757_P2.inp")

    info("model loaded.")
end

INFO: Registered handlers: Any["ELEMENT","NODE","NSET","ELSET"]
INFO: Parsing elements
INFO: 216652 elements found
INFO: Creating ELSET PISTON
INFO: Parsing elements
INFO: 24783 elements found


INFO: Creating element set BC1
INFO: Creating element set BC2
INFO: Creating element set BC3
INFO: model loaded.


5.760825 seconds (44.46 M allocations: 1.301 GB, 30.79% gc time)


In [3]:
perm = Dict{Int64, Int64}()
for (i, nid) in enumerate(keys(model["nodes"]))
    perm[nid] = i
end

elements = Dict{Int64, Element}()
for (elid, node_ids) in model["elements"]
    connectivity = Int64[perm[nid] for nid in node_ids]
    coords = Vector{Float64}[model["nodes"][nid] for nid in node_ids]
    elmap = Dict(3 => Tri3, 4 => Tet4, 6 => Tri6, 10 => Tet10)
    for (dim, eltype) in elmap
        if length(coords) == dim
            element = eltype(connectivity)
            element["geometry"] = coords
            element["id"] = elid
            elements[elid] = element
        end
    end
end

info("$(length(elements)) elements created.")

problem = ElasticityProblem()
#for elid in model["elsets"]["WRENCH"]
for elid in model["elsets"]["PISTON"]
    elements[elid]["youngs modulus"] = 210.0e3
    elements[elid]["poissons ratio"] = 0.3
    push!(problem, elements[elid])
end

traction = Vector{Float64}[[0.0, 0.0, -10.0] for i in 1:6]
#traction = Vector{Float64}[[100.0, 100.0, 100.0] for i in 1:3]
#traction = Vector{Float64}[[0.0, -100.0, 20.0] for i in 1:3]
for elid in model["elsets"]["BC1"]
    elements[elid]["displacement traction force"] = traction
    push!(problem, elements[elid])
end

bc = DirichletProblem("displacement", 3)
for elid in model["elsets"]["BC2"]
    elements[elid]["displacement"] = 0.0
    push!(bc, elements[elid])
end

solver = DirectSolver()
push!(solver, problem)
push!(solver, bc)
info("all ready for solver")

INFO: 241435 elements created.
INFO: all ready for solver


In [4]:
empty!(model)
gc()

In [5]:
call(solver, 0.0)

INFO: # of field problems: 1
INFO: # of boundary problems: 1
INFO: Starting iteration 1
INFO: Assembling boundary problems...
INFO: Assemble: 10 % done
INFO: Assemble: 20 % done
INFO: Assemble: 30 % done
INFO: Assemble: 40 % done
INFO: Assemble: 50 % done
INFO: Assemble: 60 % done
INFO: Assemble: 70 % done
INFO: Assemble: 80 % done
INFO: Assemble: 90 % done
INFO: Assemble: 100 % done
INFO: # of interface dofs: 19848
INFO: Assembling field problems...
INFO: Assembling body 1...
INFO: Assemble: 10 % done


LoadError: LoadError: InterruptException:
while loading In[5], in expression starting on line 1

In [42]:
using JuliaFEM.Core: assemble

for elid in model["elsets"]["PISTON"]
    elements[elid]["displacement"] = (0.0 => Vector{Float64}[zeros(Float64, 3) for i = 1:4])
end

function do_assembly()
    t0 = time()
    boundary_assembly = sum(map((p)->assemble(p, 0.0), solver.boundary_problems))
    boundary_dofs = unique(boundary_assembly.stiffness_matrix.I)
    info("$(length(boundary_dofs)) boundary dofs")
    field_assembly = sum(map((p)->assemble(p, 0.0), solver.field_problems))
    field_dofs = unique(field_assembly.stiffness_matrix.I)
    info("$(length(field_dofs)) field dofs")
    interior_dofs = setdiff(field_dofs, boundary_dofs)
    info("$(length(interior_dofs)) interior dofs")
    info("Assembly: ", time()-t0)

    t0 = time()
    K = sparse(field_assembly.stiffness_matrix)
    f = sparse(field_assembly.force_vector)
    @assert maximum(abs(1/2*(K + K') - K)) < 1.0e-6
    K = 1/2*(K + K')
    Kii = K[interior_dofs, interior_dofs]
    info("det = ", det(Kii))
    F = cholfact(Kii)
    Kbb = K[boundary_dofs, boundary_dofs]
    Kib = K[interior_dofs, boundary_dofs]
    fi = f[interior_dofs]
    info("Factorization: ", time()-t0)
    dim = size(K, 1)
    return dim, F, Kbb, Kib, fi, boundary_dofs, interior_dofs
end

@time dim, F, Kbb, Kib, fi, boundary_dofs, interior_dofs = do_assembly();

INFO: Assemble: 10 % done
INFO: Assemble: 20 % done
INFO: Assemble: 30 % done
INFO: Assemble: 40 % done
INFO: Assemble: 50 % done
INFO: Assemble: 60 % done
INFO: Assemble: 70 % done
INFO: Assemble: 80 % done
INFO: Assemble: 90 % done
INFO: Assemble: 100 % done
INFO: 1350 boundary dofs
INFO: Assemble: 10 % done
INFO: Assemble: 20 % done
INFO: Assemble: 30 % done
INFO: Assemble: 40 % done
INFO: Assemble: 50 % done
INFO: Assemble: 60 % done
INFO: Assemble: 70 % done
INFO: Assemble: 80 % done
INFO: Assemble: 90 % done
INFO: Assemble: 100 % done
INFO: 26367 field dofs
INFO: 25017 interior dofs
INFO: Assembly: 17.84435486793518
INFO: det = Inf
INFO: Factorization: 1.87113618850708


 19.775338 seconds (152.31 M allocations: 8.037 GB, 10.16% gc time)


In [4]:
Kc = spzeros(dim, dim)
fc = spzeros(dim, 1)

58833x1 sparse matrix with 0 Float64 entries:

In [59]:
@time begin
    nb = length(boundary_dofs)
    p = round(Int, nb/10)
    Kd = zeros(nb, nb)
    Vd = zeros(nb)
    for bi in 1:nb
        mod(bi, p) == 0 && info(round(Int, bi/nb*100), " % done.")
        C = full(F \ Kib[:, bi])
        fill!(Vd, 0.0)
        for bj in bi:nb
            d = Kib[:, bj]
            Kd[bj,bi] = dot(C[rowvals(d)], nonzeros(d))
            Vd[bj] = dot(C[rowvals(d)], nonzeros(d))
        end
        Kd[:, bi] = Vd
    end
    Kd += tril(Kd, -1)'
    Kc = spzeros(dim, dim)
    Kc[boundary_dofs, boundary_dofs] = Kbb - Kd
    info("norm(2) = ", norm(Kd))
    @assert isapprox(norm(Kd), 535881.698508461)
end

INFO: 10 % done.
INFO: 20 % done.
INFO: 30 % done.
INFO: 40 % done.
INFO: 50 % done.
INFO: 60 % done.
INFO: 70 % done.
INFO: 80 % done.
INFO: 90 % done.
INFO: 100 % done.
INFO: norm(2) = 535881.6985084609


 20.500665 seconds (13.36 M allocations: 3.063 GB, 3.34% gc time)


In [48]:
maximum(abs(1/2*(Kd + Kd') - Kd))

79868.97400906119

30% memory usage before start. 100% CPU usage. 85 % maximum memory usage.

In [5]:
function get_slices(nb, n)
#    n = 3
#    nb = 25
    kk = round(Int, collect(linspace(0, nb, n+1)))
    return [kk[j]+1:kk[j+1] for j=1:length(kk)-1]
end

get_slices (generic function with 1 method)

In [16]:
@time begin
    chunks = 10
    nb = length(boundary_dofs)
    kk = round(Int, collect(linspace(0, nb, chunks+1)))
    sl = [kk[j]+1:kk[j+1] for j=1:length(kk)-1]
    Kc = spzeros(dim, dim)
    for (k, sli) in enumerate(sl)
        b1 = boundary_dofs[sli]
        Sc = F \ Kib[:,sli]
        for slj in sl
            b2 = boundary_dofs[slj]
            Kc[b2,b1] = Kbb[slj,sli] - Kib[:,slj]'*Sc
        end
        info(round(k/chunks*100, 0), " % done")
    end
end
info(norm(Kc, 1))
@assert isapprox(norm(Kc, 1), 3.525162568997601e6)

INFO: 10.0 % done
INFO: 20.0 % done
INFO: 30.0 % done
INFO: 40.0 % done
INFO: 50.0 % done
INFO: 60.0 % done
INFO: 70.0 % done
INFO: 80.0 % done
INFO: 90.0 % done
INFO: 100.0 % done


 47

INFO: 3.525162568997601e6


.470587 seconds (251.48 k allocations: 10.678 GB, 5.06% gc time)


## Saving results to file

In [20]:
xdoc, xmodel = JuliaFEM.Postprocess.xdmf_new_model()
temporal_collection = JuliaFEM.Postprocess.xdmf_new_temporal_collection(xmodel)
grid = JuliaFEM.Postprocess.xdmf_new_grid(temporal_collection; time=0)

<Grid Name="Grid">
  <Time Value="0"/>
</Grid>


Save geometry to xdmf file

In [21]:
nnodes = length(model["nodes"])
info("number of nodes in model: $nnodes")

X = zeros(3, nnodes)
for nid in keys(model["nodes"])
    X[:, perm[nid]] = model["nodes"][nid]
end

nelements = length(model["elsets"]["PISTON"])
info("Number of elements in model: $nelements")
elmap = zeros(Int64, 5, nelements)
#elmap[1,:] = 0x0026
elmap[1,:] = 0x6
for (i, elid) in enumerate(model["elsets"]["PISTON"])
    elmap[2:end,i] = Int64[perm[nid] for nid in model["elements"][elid]]
end

INFO: number of nodes in model: 8789
INFO: Number of elements in model: 31437


In [22]:
JuliaFEM.Postprocess.xdmf_new_mesh(grid, X, elmap)

true

Save nodal data to model

In [23]:
using JuliaFEM.Core: get_connectivity

u = zeros(3, nnodes)

for element in values(elements)
    isa(element, Element{Tet4}) || continue
    connectivity = get_connectivity(element)
    field = element["displacement"](0.0)
    for (i, nid) in enumerate(connectivity)
        u[:, nid] = field[i]
    end
end
size(u)

(3,8789)

In [24]:
JuliaFEM.Postprocess.xdmf_new_field(grid, "Displacement", "nodes", u)

true

In [25]:
JuliaFEM.Postprocess.xdmf_save_model(xdoc, "/tmp/piston_8789_P1.xmf")

1525505

In [26]:
u[:, 1:10]

3x10 Array{Float64,2}:
 0.0582911   0.339694  0.431768    …   0.124351  0.346492    0.000617283
 0.0900323   0.22174   0.433814        0.140503  0.275036    0.0044649  
 0.0838928  -0.148224  0.00262483     -0.11171   0.0843743  -0.0175412  

In [27]:
minimum(u, 2)'

1x3 Array{Float64,2}:
 -0.0471355  -0.468786  -0.222571

In [28]:
maximum(u, 2)'

1x3 Array{Float64,2}:
 0.511168  0.563004  0.66328

In [30]:
piston_8789_P1_solution_norm = 38.52232721814439

38.52232721814439

In [31]:
@assert isapprox(norm(vec(u)), piston_8789_P1_solution_norm)